<a href="https://colab.research.google.com/github/jumafernandez/generacion_prompts/blob/main/semana5/RAG_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**🔵 Semana 5: ¿Qué es RAG y cómo implementarlo?**


##**1️⃣ ¿Qué es RAG?**

Retrieval-Augmented Generation (RAG) es una técnica que mejora la generación de respuestas en modelos de lenguaje combinando dos pasos:

1. **Recuperación de información relevante** desde una base de datos o documentos externos.
1. **Generación de respuestas** utilizando un modelo de lenguaje, que integra el contexto recuperado en su respuesta.
Esto permite que los modelos trabajen con información más actualizada, sin depender únicamente del conocimiento aprendido en su entrenamiento.

📌 En esta notebook vamos a implementarlo a partir de la API de OpenAI y una base de datos vectorial llamada **Chroma**.

##**2️⃣ Instalación de librerías necesarias**

Ejecuta la siguiente celda para instalar las dependencias:

In [1]:
#!pip install openai
!pip install chromadb pymupdf --quiet

import os
import sys

# Reiniciar automáticamente el entorno tras la instalación
#os.execv(sys.executable, ['python'] + sys.argv)

##**3️⃣ Configuración del entorno**

Antes de comenzar, asegúrate de configurar tu clave de API de OpenAI:

In [2]:
import os

# Configura tu clave de OpenAI
os.environ["OPENAI_API_KEY"] = ""

##**4️⃣ ¿Qué es ChromaDB y por qué lo usamos en RAG?**

ChromaDB es una base de datos vectorial que permite almacenar y recuperar información en función de su similitud semántica.

**📌 ¿Por qué usamos ChromaDB en RAG?**
- Búsqueda semántica eficiente: Permite recuperar fragmentos de documentos que sean relevantes para una consulta específica.
- Optimización en la recuperación de contexto: En lugar de depender solo del texto en el prompt, podemos buscar información adicional en documentos externos.
- Escalabilidad: Puede manejar grandes volúmenes de datos sin perder velocidad en la búsqueda.

En este caso, utilizaremos ChromaDB para indexar y buscar contenido en PDFs almacenados en un repositorio zipeado alojado en GitHub.

##**5️⃣ Cargando documentos desde un repositorio de GitHub**

Extraeremos información desde archivos PDF almacenados en un directorio clonado desde GitHub.

In [3]:
import os
import fitz  # PyMuPDF para procesar PDFs
import chromadb
from urllib import request # Import the request module for downloading files
import zipfile # Import the zipfile module for extracting zip files

In [4]:
# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="docs")

In [5]:
# Función para extraer texto de un PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = "\n".join([page.get_text() for page in doc])
        return text if text.strip() else "⚠️ Documento sin texto extraído"
    except Exception as e:
        print(f"❌ Error al leer {pdf_path}: {e}")
        return ""

# Función para descargar un archivo ZIP desde GitHub
def download_file(url, local_path):
    """Descarga un archivo desde una URL a una ruta local."""
    try:
        with request.urlopen(url) as response, open(local_path, 'wb') as out_file:
            out_file.write(response.read())
        print(f"📥 Descargado: {url} a {local_path}")
    except Exception as e:
        print(f"❌ Error al descargar el archivo: {e}")

# Función para extraer e indexar todos los PDFs del ZIP
def add_documents_from_zip(zip_url):
    # Crear un directorio para almacenar los PDFs
    local_pdf_dir = "./pdfs"
    os.makedirs(local_pdf_dir, exist_ok=True)

    # Definir la ruta del ZIP local
    zip_path = os.path.join(local_pdf_dir, "base_conocimiento.zip")

    # Descargar el ZIP
    download_file(zip_url, zip_path)

    # Extraer los PDFs
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(local_pdf_dir)
            extracted_files = zip_ref.namelist()
        print(f"📂 Archivos extraídos: {extracted_files}")
    except Exception as e:
        print(f"❌ Error al extraer el ZIP: {e}")
        return

    # Ruta correcta dentro del directorio extraído
    extracted_folder = os.path.join(local_pdf_dir, "base_conocimiento")

    # Procesar todos los PDFs extraídos
    indexed_count = 0
    if os.path.exists(extracted_folder):
        for filename in os.listdir(extracted_folder):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(extracted_folder, filename)
                print(f"🔍 Procesando: {filename}")

                text = extract_text_from_pdf(pdf_path)
                if text and text != "⚠️ Documento sin texto extraído":
                    collection.add(documents=[text], ids=[filename])
                    print(f"✅ Indexado en ChromaDB: {filename}")
                    indexed_count += 1
                else:
                    print(f"⚠️ No se pudo extraer texto de {filename}")
    else:
        print("❌ No se encontró la carpeta extraída base_conocimiento.")

    if indexed_count == 0:
        print("❌ No se indexó ningún documento en ChromaDB.")
    else:
        print(f"📂 {indexed_count} documentos fueron indexados exitosamente.")

In [6]:
# URL del ZIP en el repositorio de GitHub
pdf_zip_url = "https://github.com/jumafernandez/generacion_prompts/raw/refs/heads/main/semana5/base_conocimiento.zip"

# Agregar documentos a ChromaDB
add_documents_from_zip(pdf_zip_url)

📥 Descargado: https://github.com/jumafernandez/generacion_prompts/raw/refs/heads/main/semana5/base_conocimiento.zip a ./pdfs/base_conocimiento.zip
📂 Archivos extraídos: ['base_conocimiento/', 'base_conocimiento/566eebeb-fa42-4b2a-8143-9d6e22d7670c.pdf', 'base_conocimiento/b3094a32-663d-4c3c-ab99-bc835fa0e8dd.pdf', 'base_conocimiento/DISP_CD-CB_Nº_009_24.pdf', 'base_conocimiento/DISP_CD-CB_Nº_563_23.pdf', 'base_conocimiento/DISP_DD-CB_Nº_008_24.pdf', 'base_conocimiento/DISP_PC-CB_Nº_110_24.pdf', 'base_conocimiento/RES_HCS_Nº_162_24.pdf']
🔍 Procesando: DISP_CD-CB_Nº_563_23.pdf


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 13.3MiB/s]


✅ Indexado en ChromaDB: DISP_CD-CB_Nº_563_23.pdf
🔍 Procesando: 566eebeb-fa42-4b2a-8143-9d6e22d7670c.pdf
✅ Indexado en ChromaDB: 566eebeb-fa42-4b2a-8143-9d6e22d7670c.pdf
🔍 Procesando: RES_HCS_Nº_162_24.pdf
✅ Indexado en ChromaDB: RES_HCS_Nº_162_24.pdf
🔍 Procesando: DISP_PC-CB_Nº_110_24.pdf
✅ Indexado en ChromaDB: DISP_PC-CB_Nº_110_24.pdf
🔍 Procesando: b3094a32-663d-4c3c-ab99-bc835fa0e8dd.pdf
✅ Indexado en ChromaDB: b3094a32-663d-4c3c-ab99-bc835fa0e8dd.pdf
🔍 Procesando: DISP_CD-CB_Nº_009_24.pdf
✅ Indexado en ChromaDB: DISP_CD-CB_Nº_009_24.pdf
🔍 Procesando: DISP_DD-CB_Nº_008_24.pdf
✅ Indexado en ChromaDB: DISP_DD-CB_Nº_008_24.pdf
📂 7 documentos fueron indexados exitosamente.


##**6️⃣ Recuperar información relevante desde ChromaDB**

Para responder preguntas, primero buscamos los fragmentos más relevantes en la base de datos.

In [7]:
# Función para recuperar contexto desde ChromaDB
def retrieve_context(query, n_results=3):
    results = collection.query(query_texts=[query], n_results=n_results)
    return "\n".join(results['documents'][0]) if results['documents'] else ""

In [8]:
# Ejemplo de búsqueda
query = "¿Quién es Juan Manuel Fernandez?"
context = retrieve_context(query)

print("🔍 Contexto recuperado:\n", context)

🔍 Contexto recuperado:
  
Universidad Nacional de Luján 
REPÚBLICA ARGENTINA
“2024 – 40 años de la Reapertura de la Universidad Nacional de Luján y 30 años
del Reconocimiento Constitucional de la Autonomía Universitaria”
 
1 / 2
RESOLUCION RECTOR RR : 359 / 2024
ACTND 129/2024
LUJÁN, BUENOS AIRES
 
VISTO: La presentación efectuada por el  señor  Juan Manuel FERNÁNDEZ, mediante la
cual presenta la renuncia al cargo Nodocente, a partir del 1º de septiembre 2024, y
CONSIDERANDO:

Que el mencionado trabajador, revista en un cargo de personal Nodocente, Planta
Permanente, Director de Gestión Curricular, Categoría 2  -  Tramo Mayor del
Agrupamiento Administrativo y posee dependencia funcional en la Dirección General
de Asuntos Académicos.
Que el trabajador FERNÁNDEZ, se encuentra usufructuando licencia sin goce de
haberes por desempeño de cargo de mayor jerarquía desde el 1º de febrero de 2020,
en el marco de lo determinado por el Artículo 105 del Convenio Colectivo de Trabajo
para el Sector

##**7️⃣ Generación de respuestas con OpenAI**

Ahora usamos el contexto recuperado para generar respuestas mejoradas con OpenAI.

In [9]:
from openai import OpenAI

# Inicializar cliente de OpenAI
client = OpenAI()

In [10]:
# Función para generar respuestas usando RAG con el formato correcto de chat
def generate_response(query):
    context = retrieve_context(query)

    messages = [
        {"role": "system", "content": "Eres un asistente experto en responder preguntas basadas en el contexto recuperado."},
        {"role": "user", "content": f"Contexto: {context}\n\nPregunta: {query}\nRespuesta:"}
    ]

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages
    )

    return response.choices[0].message.content.strip()

In [11]:
# Realizar una consulta
question = "¿Quién es Juan Manuel Fernández?"

In [12]:
response = generate_response(question)

print("💡 Respuesta con RAG:\n", response)

💡 Respuesta con RAG:
 Juan Manuel Fernández es una persona con diversas responsabilidades y roles dentro de la Universidad Nacional de Luján, según la información proporcionada en los documentos aportados. En primer lugar, Juan Manuel Fernández presentó su renuncia al cargo de personal Nodocente, en la planta permanente, específicamente como Director de Gestión Curricular, efectiva a partir del 1° de septiembre de 2024. Fernández también tenía un cargo de mayor jerarquía desde el 1° de febrero de 2020, del cual usufructuaba una licencia sin goce de haberes.

Aparte de su rol nodocente, Juan Manuel Fernández ha sido nombrado como Coordinador de la Licenciatura en Sistemas de Información, desde el 1 de abril de 2024 hasta el 31 de marzo de 2026. Adicionalmente, él lidera un Proyecto de Investigación titulado "Generación de herramientas de extracción de información ambiental mediante técnicas de aprendizaje automático, a partir de bases de datos textuales", enfocado en el área de Luján, p

##**8️⃣ Comparando respuestas con y sin RAG**

Probemos la diferencia entre respuestas con y sin acceso a la base de datos.

In [15]:
# Función para generar respuestas SIN RAG (solo el modelo de OpenAI)
def generate_response_no_rag(query):
    messages = [
        {"role": "system", "content": "Eres un asistente que responde preguntas de forma precisa."},
        {"role": "user", "content": query}
    ]

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages
    )

    return response.choices[0].message.content.strip()

In [16]:
response_no_rag = generate_response_no_rag(question)

print("🔵 Respuesta SIN RAG:\n", response_no_rag)

🔵 Respuesta SIN RAG:
 Necesitaría más información o contexto para poder proporcionar detalles específicos sobre Juan Manuel Fernández, ya que este nombre es bastante común y podría referirse a varias personas en diferentes campos como política, deportes, arte, entre otros. Si puedes proporcionar detalles adicionales o especificar el ámbito en el que Juan Manuel Fernández es conocido, estaré encantado de ayudarte con una respuesta más precisa.
